In [1]:
import os
from pathlib import Path
from tqdm.auto import tqdm

from model import ViTEmbedding
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

class ImageDataset:
    def __init__(self, images_dir, split):
        self.images_dir = images_dir
        self.image_list = list(Path(images_dir).glob('*.png'))
        self.image_list = sorted(self.image_list,
                                 key=lambda x: int(x.name.replace('.png','').replace(f'CLEVR_{split}_','')))
        self.transforms = transforms.ToTensor()

    def __getitem__(self, idx):
        image_path = self.image_list[idx]
        image_tensor = self.transforms(self.load_image(image_path))
        return image_tensor

    def load_image(self, image_path):
        return Image.open(image_path).convert('RGB')

    def __len__(self):
        return len(self.image_list)

base_dir = '/workspace1/fidelrio/CLEVR_CoGenT_v1.0/'
base_dir2 = '/mnt/ialabnas/datasets/CLEVR_CoGenT_v1.0/'
    
device = torch.device('cuda')

vit = ViTEmbedding().to(device)

# for split in ['trainA', 'valA', 'valB', 'testA', 'testB']:
for split in ['valA', 'valB', 'testA', 'testB']:
    images_dir = f'{base_dir}/images/{split}'
    dataset = ImageDataset(images_dir, split)

    num_workers = int(os.environ.get("SLURM_CPUS_PER_TASK", 4))
    loader = DataLoader(dataset, batch_size=32, num_workers=num_workers, pin_memory=True)

    all_outs = []
    for images in tqdm(loader):
        images = images.to(device)
        all_outs.append(vit(images).detach().cpu())

    all_feats = torch.cat(all_outs)

    image_name_to_idx = {image_path.name: idx for idx, image_path in enumerate(dataset.image_list)}

    torch.save({
            'image_name_to_idx': image_name_to_idx,
            'image_features': all_feats,
        },
        f'{base_dir}/images/{split}-vit.pt')
    
    torch.save({
            'image_name_to_idx': image_name_to_idx,
            'image_features': all_feats,
        },
        f'{base_dir2}/images/{split}-vit.pt')

2024-05-16 14:37:38.072327: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-8.0/lib64/
2024-05-16 14:37:38.072364: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a Ber

  0%|          | 0/469 [00:00<?, ?it/s]

  0%|          | 0/469 [00:01<?, ?it/s]

0it [00:01, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/home/fidelrio/.pyenv/versions/3.7.5/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/home/fidelrio/.pyenv/versions/3.7.5/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/home/fidelrio/.pyenv/versions/3.7.5/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/fidelrio/.pyenv/versions/3.7.5/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/fidelrio/.pyenv/versions/3.7.5/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/fidelrio/.pyenv/versions/3.7.5/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: se

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:50                                                                                   │
│                                                                                                  │
│   47 │   │   images = images.to(device)                                                          │
│   48 │   │   all_outs.append(vit(images).detach().cpu())                                         │
│   49 │                                                                                           │
│ ❱ 50 │   all_feats = torch.cat(all_outs)                                                         │
│   51 │                                                                                           │
│   52 │   image_name_to_idx = {image_path.name: idx for idx, image_path in enumerate(dataset.i    │
│   53                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NotImplementedError: There were no tensor arguments to this function (e.g., you passed an empty list of Tensors), 
but no fallback function is registered for schema aten::_cat.  This usually means that this function requires a 
non-empty list of Tensors, or that you (the operator writer) forgot to register a fallback function.  Available 
functions are [CPU, CUDA, QuantizedCPU, BackendSelect, Python, Named, Conjugate, Negative, ZeroTensor, 
ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradLazy, AutogradXPU, AutogradMLC, 
AutogradHPU, AutogradNestedTensor, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, Tracer, 
AutocastCPU, Autocast, Batched, VmapMode, Functionalize].

CPU: registered at aten/src/ATen/RegisterCPU.cpp:21063 [kernel]
CUDA: registered at aten/src/ATen/RegisterCUDA.cpp:29726 [kernel]
QuantizedCPU: registered at aten/src/ATen/RegisterQuantizedCPU.cpp:1258 [kernel]
BackendSelect: fallthrough registered at ../aten/src/ATen/core/BackendSelectFallbackKernel.cpp:3 [backend fallback]
Python: registered at ../aten/src/ATen/core/PythonFallbackKernel.cpp:47 [backend fallback]
Named: registered at ../aten/src/ATen/core/NamedRegistrations.cpp:7 [backend fallback]
Conjugate: registered at ../aten/src/ATen/ConjugateFallback.cpp:18 [backend fallback]
Negative: registered at ../aten/src/ATen/native/NegateFallback.cpp:18 [backend fallback]
ZeroTensor: registered at ../aten/src/ATen/ZeroTensorFallback.cpp:86 [backend fallback]
ADInplaceOrView: fallthrough registered at ../aten/src/ATen/core/VariableFallbackKernel.cpp:64 [backend fallback]
AutogradOther: registered at ../torch/csrc/autograd/generated/VariableType_3.cpp:11380 [autograd kernel]
AutogradCPU: registered at ../torch/csrc/autograd/generated/VariableType_3.cpp:11380 [autograd kernel]
AutogradCUDA: registered at ../torch/csrc/autograd/generated/VariableType_3.cpp:11380 [autograd kernel]
AutogradXLA: registered at ../torch/csrc/autograd/generated/VariableType_3.cpp:11380 [autograd kernel]
AutogradLazy: registered at ../torch/csrc/autograd/generated/VariableType_3.cpp:11380 [autograd kernel]
AutogradXPU: registered at ../torch/csrc/autograd/generated/VariableType_3.cpp:11380 [autograd kernel]
AutogradMLC: registered at ../torch/csrc/autograd/generated/VariableType_3.cpp:11380 [autograd kernel]
AutogradHPU: registered at ../torch/csrc/autograd/generated/VariableType_3.cpp:11380 [autograd kernel]
AutogradNestedTensor: registered at ../torch/csrc/autograd/generated/VariableType_3.cpp:11380 [autograd kernel]
AutogradPrivateUse1: registered at ../torch/csrc/autograd/generated/VariableType_3.cpp:11380 [autograd kernel]
AutogradPrivateUse2: registered at ../torch/csrc/autograd/generated/VariableType_3.cpp:11380 [autograd kernel]
AutogradPrivateUse3: registered at ../torch/csrc/autograd/generated/VariableType_3.cpp:11380 [autograd kernel]
Tracer: re